In [10]:
from shapely.geometry import box

input_folder = "/local_data/cosmiq/wdata/achadda/rareplanes-public/testOriginal/geojson_aircraft_tiled"
output_folder = "/local_data/cosmiq/wdata/achadda/rareplanes-public/testOriginal/geojson_aircraft_bbox"
os.makedirs(output_folder,exist_ok=True)
os.chdir(input_folder)
geojsons = glob.glob("*.geojson")
for geojson in tqdm(geojsons):
    gdf = gpd.read_file(geojson)
    box_geoms= []
    for _, row in gdf.iterrows():
        x = row.geometry.bounds
        bbox = box(x[0], x[1], x[2], x[3])
        box_geoms.append(bbox)
    gdf['box_geom'] = box_geoms
    gdf = gdf.drop('geometry',axis=1)
    gdf = gpd.GeoDataFrame(gdf, geometry='box_geom')
    out_name=os.path.join(output_folder, geojson)
    #print(gdf.head(10)) 
    gdf.to_file(out_name, driver='GeoJSON', encoding='utf-8')

100%|██████████| 2710/2710 [01:00<00:00, 44.61it/s]


In [12]:
import argparse
import geopandas as gpd
import pandas as pd
import os
import glob
from tqdm import tqdm


def count_unique_index(df, by):
    return df.groupby(by).size().reset_index().rename(columns={0: 'count'})

def create_custom_classes(all_annotations_geojson, geojson_dir, output_path, category_attributes):
    """ parse the geojson files and create custom classes based upon
    unique variatons of the RarePlanes attributes.
        -all_annotations_geojson (str): The path to the
        `RarePlanes_Public_All_Annotations.geojson` file.
        - geojson_dir (str): directory containing the geojson files
        for individual images or tiles
        - output_path (str): directory to output the customized geojsons. Need to provide the absolute path.
        - category_attributes (list): A list of attributes to combine
        to create a custom class.  Choose any combintaion of the following:
        ['role','num_engines', 'propulsion', 'canards', 'num_tail_fins',
       'wing_position', 'wing_type', 'faa_wingspan_class']
    :returns
        - new geojsons with a custom_id for each combination of unique
        attributes.
        -A lookup table for each classes custom_id.
    """
    os.makedirs(output_path, exist_ok=True)
    gdf = gpd.read_file(all_annotations_geojson)
    lookup_gdf = count_unique_index(gdf, category_attributes)
    lookup_gdf['custom_id'] = list(range(1, len(lookup_gdf) + 1))
    lookup_gdf.drop(columns=['count'], inplace=True)
    lookup_gdf.to_csv(os.path.join(output_path, "custom_class_lookup.csv"))
    os.chdir(geojson_dir)
    geojsons = glob.glob("*.geojson")
    for geojson in tqdm(geojsons):
        gdf = gpd.read_file(geojson)
        gdf = pd.merge(gdf, lookup_gdf, on=category_attributes, how='left')
        gdf.to_file(os.path.join(output_path, geojson), driver="GeoJSON", encoding='utf-8')

In [13]:
all_annotations_geojson = '/local_data/cosmiq/wdata/achadda/rareplanes-public/RarePlanes_Public_All_Annotations.geojson'
#geojson_dir_train = '/local_data/cosmiq/wdata/achadda/rareplanes-public/trainOriginal/geojson_aircraft_tiled'
geojson_dir_test = '/local_data/cosmiq/wdata/achadda/rareplanes-public/testOriginal/geojson_aircraft_bbox'

output_path_test_one = '/local_data/cosmiq/wdata/achadda/rareplanes-public/testOriginal/yolo_class_one_bbox'
output_path_test_two = '/local_data/cosmiq/wdata/achadda/rareplanes-public/testOriginal/yolo_class_two_bbox'
output_path_test_three = '/local_data/cosmiq/wdata/achadda/rareplanes-public/testOriginal/yolo_class_three_bbox'
output_path_test_four = '/local_data/cosmiq/wdata/achadda/rareplanes-public/testOriginal/yolo_class_four_bbox'
output_path_test_five = '/local_data/cosmiq/wdata/achadda/rareplanes-public/testOriginal/yolo_class_five_bbox'

class_one = ['num_engines', 'propulsion']
class_two = ['role']
class_three = ['num_tail_fins','wing_position', 'wing_type']
class_four = ['role','num_engines', 'propulsion', 'canards', 'num_tail_fins','wing_position', 'wing_type', 'faa_wingspan_class']
class_five = ['num_engines']

create_custom_classes(all_annotations_geojson, geojson_dir_test, output_path_test_one, class_one)
create_custom_classes(all_annotations_geojson, geojson_dir_test, output_path_test_two, class_two)
create_custom_classes(all_annotations_geojson, geojson_dir_test, output_path_test_three, class_three)
create_custom_classes(all_annotations_geojson, geojson_dir_test, output_path_test_four, class_four)
create_custom_classes(all_annotations_geojson, geojson_dir_test, output_path_test_five, class_five)

100%|██████████| 2710/2710 [01:02<00:00, 43.03it/s]
